## Обучение TabNet

In [1]:
!pip install pytorch-tabnet neptune-client

     |████████████████████████████████| 102kB 3.8MB/s 
     |████████████████████████████████| 829kB 8.1MB/s 
     |████████████████████████████████| 61kB 4.8MB/s 
     |████████████████████████████████| 204kB 14.2MB/s 
     |████████████████████████████████| 163kB 17.6MB/s 
     |████████████████████████████████| 133kB 18.3MB/s 
     |████████████████████████████████| 71kB 4.3MB/s 
     |████████████████████████████████| 71kB 4.8MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.4.129-py2.py3-none-any.whl size=152627 sha256=aafb7effa327fa7a95e40ed005536f8aeb59ed5261041782f8a69457becc3571
  Stored in directory: /root/.cache/pip/wheels/83/36/10/50fcc95c5ad563e8b5394819ba3accad734c2208be90919dab
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=b4e46c2fa113f95203b0f55d9a1bedf93f481998cb2192a354cde11dc774cbb6
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for 

In [2]:
import os
import numpy as np
import torch
import random
import pickle

from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor
import neptune

In [76]:
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [77]:
salary = np.load('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/salary_scaled.npy')

In [78]:
salary = salary.reshape(-1, 1)

###Подгрузим наши индексы и массивы переменных

In [112]:
X = np.concatenate((
    np.load('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/responsibilities_vectors.npy'),
    np.load('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/industry_label.npy'),
    np.load('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/employement_type_label.npy'),
    np.load('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/education_type_label.npy'),
    np.load('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/cluster_label.npy')
), axis=1)

In [98]:
#X = np.load('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/responsibilities_vectors.npy')

In [89]:
import pandas as pd

In [91]:
#X_df = pd.DataFrame({'responsibilities_vectors': X[:, 0], 'industry_label': X[:, 1]},
#                    {'employement_type_label': X[:, 2], 'education_type_label': X[:, 3]},
#                    {'cluster_label': X[:, 4]})

In [80]:
with open('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/industry2id.pickle', 'rb') as handle:
    industry2id = pickle.load(handle)
with open('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/employement_type2id.pickle', 'rb') as handle:
    employement_type2id = pickle.load(handle)
with open('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input/cluster2id.pickle', 'rb') as handle:
    cluster2id = pickle.load(handle)

In [81]:
#cat_dims = [len(industry2id), len(employement_type2id), len(cluster2id),]

In [11]:
PARAMS = {
    'description': 'TabNetL'
}
neptune.init('gleb-balchidi/pandemic-dh', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiMGIzZWQ5MjktNjFlMS00ZjljLWIyMWEtYTc2M2RkMzJlZWVmIn0=')
neptune.create_experiment(name='SalaryTabNet2',params=PARAMS)

https://ui.neptune.ai/gleb-balchidi/pandemic-dh/e/PAN-3


Experiment(PAN-3)

In [113]:
#инициализация модели
clf = TabNetRegressor(
    verbose=10,
    optimizer_fn=torch.optim.AdamW,
    optimizer_params=dict(lr=2e-2),
    scheduler_params = {"gamma": 0.95,
                     "step_size": 20},
    scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15,
    seed=seed,
    clip_value=2.,
    #cat_idxs=[768, 769, 770, 771],
    #cat_dims=cat_dims,
    #cat_emb_dim=[1, 1, 1, 100],

    n_d=64,
    n_a=64,
    n_steps=5,
    gamma=1.5,
    n_independent=2,
    n_shared=2,
    lambda_sparse=1e-4,
    momentum=0.3,
    
)

Device used : cpu


In [170]:
clf.fit(
    X[:245016],
    salary[:245016],
    eval_set=[(X[245016:], salary[245016:])],
    max_epochs=10,
    loss_fn=torch.nn.L1Loss(),
    batch_size=5104,
    virtual_batch_size=256,
    patience=200,
    drop_last=False,
    )

epoch 0  | loss: 0.55732 | val_0_mse: 1.0539  |  0:09:11s
Stop training because you reached max_epochs = 10 with best_epoch = 9 and best_val_0_mse = 1.03008
Best weights from best epoch are automatically used!


##Спустя несколько эпох, оказалось, что ошибка не убывает значительно, решено было оставить 10-ть эпох

In [171]:
X_test = np.concatenate((
    np.load('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input_test/responsibilities_vectors.npy'),
    np.load('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input_test/industry_label.npy'),
    np.load('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input_test/employement_type_label.npy'),
    np.load('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input_test/education_type_label.npy'),
    np.load('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input_test/cluster_label.npy')
), axis=1)

In [172]:
preds = clf.predict(X_test)

In [173]:
preds

array([[-0.17404644],
       [-0.31982848],
       [-0.23252247],
       ...,
       [-0.31982845],
       [-0.21896262],
       [-0.31982845]], dtype=float32)

In [174]:
ids = np.load('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input_test/id_array.npy')

In [175]:
df = pd.DataFrame(ids, columns=['id'])
df['salary'] = preds

In [176]:
df['salary'] = preds

In [146]:
df.to_csv('tab_net_salary.csv', sep=',', index=False)

In [177]:
orig = np.load('/content/drive/MyDrive/Colab Notebooks/Pandemic_data_hack/input_train/salary_orig.npy')

In [178]:
def get_salary(scaled, orig = orig) -> Optional:
    """Переводит нормализованное значение зарплаты в реальное
    """
    return (scaled * orig.std(axis=0)) + orig.mean(axis=0)

In [179]:
get_salary(df['salary'][3])

33452.69261516679

In [180]:
df['salary'] = df['salary'].apply(get_salary)

In [182]:
df.head()

,id,salary
0,2,34787.244344
1,6,29632.561369
2,7,32719.600048
3,11,33452.692615
4,16,37023.204494


In [183]:
df.to_csv('tab_net-salary_1.csv', sep=',', index=False)